In [1]:
import numpy as np
import pandas as pd
import gc
gc.collect()



path = './data/'
player_ext = 'Players_2019.csv'
event_ext = 'Events_2019.csv'
teams_ext = 'Teams.csv'

events = pd.read_csv(path+event_ext)
players = pd.read_csv(path+player_ext)

In [2]:
events['GameID'] = events['Season'].astype(str) + events['DayNum'].astype(str) + events['WTeamID'].astype(str) + events['LTeamID'].astype(str)
events['OT'] = (events['ElapsedSeconds'] - 2400)/300
events['OT'] = np.where(events['OT'] < 0, 0, np.ceil(events['OT'])).astype(int)
events['TimeRemain'] = 2400 + (events['OT'] * 300) - events['ElapsedSeconds']




In [3]:
one_point = ['made1_free']
two_point = ['made2_dunk','made2_jump','made2_lay','made2_tip']
three_point = ['made3_jump']
rebound = ['reb_def','reb_dead','reb_off']
assist = ['assist']
block = ['block']
turnover = ['turnover']
steal = ['steal']

fantasy_events = one_point + two_point + three_point + rebound + assist + block + turnover + steal

fp = events.loc[events['EventType'].isin(fantasy_events)]

fp.loc[fp['EventType'].isin(two_point), 'EventType'] = '2PT'
fp.loc[fp['EventType'].isin(rebound), 'EventType'] = 'Reb'

pgb = fp.groupby(['EventPlayerID','EventType'])['EventID'].count().unstack(level=-1).reset_index()
# pgb = pgb.rename(columns={'EventPlayerID':'PlayerID'})
pgb = pgb.fillna(0)


tgb = fp.groupby(['EventTeamID','EventType'])['EventID'].count().unstack(level=-1).reset_index()

# tgb = tgb.rename(columns={'FGA':'TFGA','FTA':'TFTA','turnover':'TTO','EventTeamID':'TeamID'})
tgb.head()

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


EventType,EventTeamID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover
0,1101,525,994,424,74,400,210,232,338
1,1102,542,1104,412,56,330,225,149,409
2,1103,467,1194,370,97,360,275,174,369
3,1104,594,1356,401,154,482,233,152,449
4,1105,535,1077,379,48,277,180,221,477


In [4]:
list(events)

['EventID',
 'Season',
 'DayNum',
 'WTeamID',
 'LTeamID',
 'WPoints',
 'LPoints',
 'ElapsedSeconds',
 'EventTeamID',
 'EventPlayerID',
 'EventType',
 'GameID',
 'OT',
 'TimeRemain']

In [5]:
# need total games played
tgp = events.groupby(['EventTeamID','GameID'])['OT'].max().reset_index()

ot = tgp.groupby(['EventTeamID'])['OT'].sum().reset_index()

tgp = tgp.groupby(['EventTeamID'])['GameID'].count().reset_index()

tgp = tgp.rename(columns={'GameID':'GameCount'})
ot = ot.rename(columns={'OT':'OTCount'})

ot.OTCount.median()

2.0

In [6]:
# player games played
pgp = events.groupby(['EventPlayerID','GameID'])['OT'].max().reset_index()
pgp = pgp.groupby(['EventPlayerID'])['GameID'].count().reset_index()
pgp = pgp.rename(columns={'GameID':'GameCount'})
pgp.head()

,EventPlayerID,GameCount
0,653508,21
1,653509,13
2,653510,26
3,653511,29
4,653512,29


In [7]:
tgb = pd.merge(tgb, tgp, how='left',on=['EventTeamID','EventTeamID'])
tgb.head()

,EventTeamID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover,GameCount
0,1101,525,994,424,74,400,210,232,338,29
1,1102,542,1104,412,56,330,225,149,409,31
2,1103,467,1194,370,97,360,275,174,369,31
3,1104,594,1356,401,154,482,233,152,449,33
4,1105,535,1077,379,48,277,180,221,477,31


In [8]:
tgb['Team_FPTS'] = 2*tgb['2PT'] + 1.25 * tgb['Reb'] + 1.5 * tgb['assist'] + 2 * tgb['block'] + tgb['made1_free'] + 3.5*tgb['made3_jump'] + 2*tgb['steal'] - 0.5*tgb['turnover']
# add small OT factor
tgb['TPPG'] = tgb['Team_FPTS']/(tgb['GameCount']+0.25)

tgb['TPPG'] = tgb['TPPG'].round(2)
tgb = tgb[['EventTeamID','TPPG']]
tgb.head()


,EventTeamID,TPPG
0,1101,154.07
1,1102,140.96
2,1103,149.17
3,1104,155.47
4,1105,134.12


In [9]:
pgb = pd.merge(pgb, pgp, how='left',on=['EventPlayerID','EventPlayerID'])


In [10]:
pgb['Player_FPTS'] = 2*pgb['2PT'] + 1.25 * pgb['Reb'] + 1.5 * pgb['assist'] + 2 * pgb['block'] + pgb['made1_free'] + 3.5*pgb['made3_jump'] + 2*pgb['steal'] - 0.5*pgb['turnover']
# add small OT factor
pgb['FPPG'] = pgb['Player_FPTS']/(pgb['GameCount']+0.25)

pgb['FPPG'] = pgb['FPPG'].round(2)
pgb = pgb.sort_values(by=['FPPG'],ascending=False)
pgb.head()



,EventPlayerID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover,GameCount,Player_FPTS,FPPG
2620,656216,188.0,164.0,290.0,24.0,199.0,49.0,47.0,155.0,29,1451.00,49.61
892,654429,322.0,361.0,64.0,71.0,126.0,7.0,48.0,105.0,32,1527.25,47.36
3458,657082,203.0,363.0,55.0,30.0,186.0,64.0,24.0,68.0,31,1426.25,45.64
554,654080,142.0,143.0,85.0,9.0,227.0,124.0,43.0,72.0,29,1319.25,45.10
1834,655408,166.0,256.0,163.0,30.0,126.0,45.0,63.0,90.0,30,1321.00,43.67


In [11]:
players = players.drop(columns=['Season'])

#rename
pgb = pgb.rename(columns={'EventPlayerID':'PlayerID'})
tgb = tgb.rename(columns={'EventTeamID':'TeamID'})

pgb = pd.merge(left=pgb, right=players, how='left', on=['PlayerID','PlayerID'])
pgb = pd.merge(left=pgb, right=tgb, how='left', on=['TeamID','TeamID'])
pgb.head()



,PlayerID,2PT,Reb,assist,block,made1_free,made3_jump,steal,turnover,GameCount,Player_FPTS,FPPG,TeamID,PlayerName,TPPG
0,656216,188.0,164.0,290.0,24.0,199.0,49.0,47.0,155.0,29,1451.00,49.61,1293,JA_MORANT,177.03
1,654429,322.0,361.0,64.0,71.0,126.0,7.0,48.0,105.0,32,1527.25,47.36,1169,LAMINE_DIANE,164.98
2,657082,203.0,363.0,55.0,30.0,186.0,64.0,24.0,68.0,31,1426.25,45.64,1355,MIKE_DAUM,171.93
3,654080,142.0,143.0,85.0,9.0,227.0,124.0,43.0,72.0,29,1319.25,45.10,1144,CHRIS_CLEMONS,152.09
4,655408,166.0,256.0,163.0,30.0,126.0,45.0,63.0,90.0,30,1321.00,43.67,1236,JOHN_KONCHAR,169.50


In [12]:
fp = pgb[['TeamID','PlayerID','PlayerName','FPPG','TPPG']]

teams_ext = 'Teams.csv'
teams = pd.read_csv(path+teams_ext)
teams = teams[['TeamID','TeamName']]

fp = pd.merge(fp, teams, how='left',on=['TeamID','TeamID'])

fp = fp[['TeamName','PlayerName','FPPG']]
fp.head()

,TeamName,PlayerName,FPPG
0,Murray St,JA_MORANT,49.61
1,CS Northridge,LAMINE_DIANE,47.36
2,S Dakota St,MIKE_DAUM,45.64
3,Campbell,CHRIS_CLEMONS,45.10
4,IPFW,JOHN_KONCHAR,43.67


In [13]:
needed_teams = ['Clemson','Virginia Tech','Duke','Kansas','Fresno St','Oregon','Georgia Tech','NC State',
                'Michigan St','Kentucky','Nevada','Utah',"St Mary's CA",'Wisconsin']
needed_teams = ['North Florida']
fp = fp.loc[fp.TeamName.isin(needed_teams)]
print(len(fp.TeamName.unique()))


1


In [22]:
rosters = pd.read_csv('nf_roster.csv')
rosters = rosters.drop(columns=['Unnamed: 0','number'])
rosters['Team'] = 'North Florida'
rosters.head()


,name,position,height,weight,class,hometown,Team
0,Emmanuel Adedoyin,G,"6' 2""",161 lbs,FR,"Orlando, FL",North Florida
1,Emmanuel Lambright,G,"6' 6""",210 lbs,SO,"Sarasota, FL",North Florida
2,Wajid Aminu,F,"6' 7""",197 lbs,SR,"Stone Mountain, GA",North Florida
3,J.T. Escobar,G,"6' 2""",186 lbs,SR,"Tallahassee, FL",North Florida
4,Ezekiel Balogun,F,"6' 6""",238 lbs,SR,"Lagos, Nigeria",North Florida


In [23]:
team_trans = {
    'Michigan State':'Michigan St',
    "Saint Mary's":"St Mary's CA",
    "Fresno State":'Fresno St'
}

rosters = rosters.replace({"Team": team_trans})

print(rosters.Team.unique())
print(fp.TeamName.unique())


['North Florida']
['North Florida']


In [24]:
fp.loc[:,'PlayerName'] = fp['PlayerName'].str.replace("_"," ")
fp.loc[:,'PlayerName'] = fp['PlayerName'].str.title()

fp.head()


,TeamName,PlayerName,FPPG
82,North Florida,Noah Horchler,33.71
398,North Florida,Wajid Aminu,26.57
600,North Florida,Ivan Gandia-Rosa,24.32
677,North Florida,Garrett Sams,23.64
1603,North Florida,J.T. Escobar,15.83


In [25]:
roster_to_fp = {
    "Alex O'Connell":'Alex Oconnell',
    "C.J. Bryce":'Cj Bryce',
    "D'Mitrik Trice":"Dmitrik Trice",
    "DJ Funderburk":'Derek Funderburk',
    "David McCormack":'David Mccormack',
    "EJ Montgomery":'Ej Montgomery',
    "James Banks III":'James Banks Iii',
    "John Newman III":'John Newman Iii',
    "Marcus Bingham Jr.":'Marcus Bingham Jr',
    "P.J. Horne":'Pj Horne',
    "Walt McGrory":'Walt Mcgrory',
    "J.T. Escobar":'Jt Escobar'
}

fp_to_roster = dict(map(reversed, roster_to_fp.items()))
fp = fp.replace({"PlayerName": fp_to_roster})


In [26]:
# ### check for similar names to match 
roster_names = list(rosters.name.values)
fp_names = list(fp.PlayerName.values)

print("Use minus roster")
fdiff = list(set(fp_names)-set(roster_names))
fdiff.sort()
print(fdiff)

print("Roster minus use")
rdiff = list(set(roster_names)-set(fp_names))
rdiff.sort()
print(rdiff)

import difflib

for rword in rdiff:
    print(rword, difflib.get_close_matches(rword,fdiff))

Use minus roster
['Brian Coffey Ii', 'Cam Copeland', 'Noah Horchler', 'Team']
Roster minus use
['Christian Schach', 'Dorian James', 'Emmanuel Adedoyin', 'Jose Placer', 'Josh Endicott']
Christian Schach []
Dorian James []
Emmanuel Adedoyin []
Jose Placer []
Josh Endicott []


In [27]:
# not good enough for just names, need name and team

rosters['NameID'] = rosters['Team'] + ' ' + rosters['name']
fp['NameID'] = fp['TeamName'] + ' ' + fp['PlayerName']

print(fp.head())

           TeamName        PlayerName   FPPG                          NameID
82    North Florida     Noah Horchler  33.71     North Florida Noah Horchler
398   North Florida       Wajid Aminu  26.57       North Florida Wajid Aminu
600   North Florida  Ivan Gandia-Rosa  24.32  North Florida Ivan Gandia-Rosa
677   North Florida      Garrett Sams  23.64      North Florida Garrett Sams
1603  North Florida      J.T. Escobar  15.83      North Florida J.T. Escobar


In [28]:
roster_names = list(rosters.NameID.values)

fp['Graduated'] = np.where(fp['NameID'].isin(roster_names),"Still On Roster","Lost")


In [29]:
pos = rosters[['NameID','position']]

fp = pd.merge(fp, pos, how='left',on=['NameID','NameID'])
fp.head()

,TeamName,PlayerName,FPPG,NameID,Graduated,position
0,North Florida,Noah Horchler,33.71,North Florida Noah Horchler,Lost,NaN
1,North Florida,Wajid Aminu,26.57,North Florida Wajid Aminu,Still On Roster,F
2,North Florida,Ivan Gandia-Rosa,24.32,North Florida Ivan Gandia-Rosa,Still On Roster,G
3,North Florida,Garrett Sams,23.64,North Florida Garrett Sams,Still On Roster,G
4,North Florida,J.T. Escobar,15.83,North Florida J.T. Escobar,Still On Roster,G


In [30]:
fgb = fp.groupby(['TeamName','Graduated'])['FPPG'].sum().unstack(-1).reset_index()
fgb.sort_values(by="Still On Roster", ascending=False)


Graduated,TeamName,Lost,Still On Roster
0,North Florida,47.35,128.05


In [31]:
returning = fp.loc[fp.Graduated=="Still On Roster"]
returning = returning.drop(columns=['Graduated'])

fgb = fgb.rename(columns={'Lost':'Available FPPG'})
# fgb = fgb.drop(columns=['Still On Roster'])

returning = pd.merge(returning, fgb, how='left',on=['TeamName','TeamName'])
returning = returning.sort_values(by=['FPPG'],ascending=False)
returning = returning.drop(columns=['NameID'])
print(returning.head(50))

        TeamName          PlayerName   FPPG position  Available FPPG  \
0  North Florida         Wajid Aminu  26.57        F           47.35   
1  North Florida    Ivan Gandia-Rosa  24.32        G           47.35   
2  North Florida        Garrett Sams  23.64        G           47.35   
3  North Florida        J.T. Escobar  15.83        G           47.35   
4  North Florida  Carter Hendricksen  11.35        F           47.35   
5  North Florida            Trip Day   9.49        F           47.35   
6  North Florida     Ezekiel Balogun   9.23        F           47.35   
7  North Florida      Ryan Burkhardt   5.62        G           47.35   
8  North Florida  Emmanuel Lambright   2.00        G           47.35   

   Still On Roster  
0           128.05  
1           128.05  
2           128.05  
3           128.05  
4           128.05  
5           128.05  
6           128.05  
7           128.05  
8           128.05  


In [32]:
returning.loc[returning['position']=='C', 'position'] = 'F'
lost = fgb[['TeamName','Available FPPG']]
lost = lost.rename(columns={'Available FPPG':'Lost'})

pos_gb = returning.groupby(['TeamName','position'])['FPPG'].sum().unstack(-1).reset_index()
pos_gb = pd.merge(pos_gb, lost, how='left',on=['TeamName','TeamName'])
pos_gb = pos_gb.sort_values(by=['Lost'],ascending=False)
pos_gb = pos_gb.fillna(0)
pos_gb

,TeamName,F,G,Lost
0,North Florida,56.64,71.41,47.35
